In [ ]:

!pip install pyspark


In [4]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
schema1 = T.StructType([
                T.StructField("id", T.IntegerType(), True),
                T.StructField("timestamp", T.IntegerType(), True),
                T.StructField("type", T.StringType(), True),
                T.StructField("page_id", T.IntegerType(), True),
                T.StructField("tag", T.StringType(), True),
                T.StructField("sign", T.BooleanType(), True)])


In [6]:
data1 =    [(234, 1670588026, "visit", 34, 'fun', False),
           (234, 1670588037, "scroll", 34, 'fun', False),
           (235, 1670588126, "visit", 45, 'news', False),
           (235, 1670588141, "click", 49, 'today', False),
           (235, 1670588167, "scroll", 49, 'today', True),
           (235, 1670588243, "move", 13, 'sport', False),
           (236, 1670588286, "visit", 1, 'main', False),
           (236, 1670588331, "scroll", 1, 'main', False),
           (237, 1670618426, "visit", 45, 'news', False),
           (237, 1670618479, "scroll", 45, 'news', False),
           (237, 1670618541, "click", 25, 'kino', False),
           (237, 1670618592, "scroll", 25, 'kino', True),
           (237, 1670618613, "move", 1, 'main', False),  
           (238, 1670618821, "visit", 34, 'fun', False),
           (238, 1670618877, "scroll", 34, 'fun', False),
           (238, 1670618991, "click", 25, 'kino', False),
           (239, 1670619026, "visit", 1, 'main', False),
           (239, 1670619051, "click", 49, 'today', False),
           (240, 1670769167, "visit", 1, 'main', True),
           (240, 1670769243, "move", 13, 'sport', False),
           (240, 1670769283, "move", 1, 'main', False),
           (240, 1670769386, "visit", 45, 'news', False),
           (240, 1670769311, "scroll", 45, 'news', False),
           (241, 167769516, "visit", 45, 'news', False),
           (241, 1670769579, "scroll", 45, 'news', False),
           (241, 1670769741, "click", 34, 'fun', False),
           (241, 1670769822, "scroll", 34, 'fun', True),
           (241, 1670769963, "move", 1, 'main', False)] 



In [7]:
df_log1 = spark.createDataFrame(data = data1, schema = schema1)

In [8]:
df_log1.show()

+---+----------+------+-------+-----+-----+
| id| timestamp|  type|page_id|  tag| sign|
+---+----------+------+-------+-----+-----+
|234|1670588026| visit|     34|  fun|false|
|234|1670588037|scroll|     34|  fun|false|
|235|1670588126| visit|     45| news|false|
|235|1670588141| click|     49|today|false|
|235|1670588167|scroll|     49|today| true|
|235|1670588243|  move|     13|sport|false|
|236|1670588286| visit|      1| main|false|
|236|1670588331|scroll|      1| main|false|
|237|1670618426| visit|     45| news|false|
|237|1670618479|scroll|     45| news|false|
|237|1670618541| click|     25| kino|false|
|237|1670618592|scroll|     25| kino| true|
|237|1670618613|  move|      1| main|false|
|238|1670618821| visit|     34|  fun|false|
|238|1670618877|scroll|     34|  fun|false|
|238|1670618991| click|     25| kino|false|
|239|1670619026| visit|      1| main|false|
|239|1670619051| click|     49|today|false|
|240|1670769167| visit|      1| main| true|
|240|1670769243|  move|     13|s

In [9]:
df_log1.groupby("id").agg(F.count("*").alias("event")).orderBy("event", ascending = False).show(5)

+---+-----+
| id|event|
+---+-----+
|241|    5|
|237|    5|
|240|    5|
|235|    4|
|238|    3|
+---+-----+
only showing top 5 rows



In [10]:
users = df_log1.groupby("id").agg(F.count("*")).count()

sign = df_log1.where((F.col('sign')=='True')).count()
percent = sign/users*100

print("Convertion =", percent)


Convertion = 50.0


In [31]:
df_log1.where((F.col('type')=='click')).groupby("page_id").agg(F.count("*").alias("event")).orderBy("event", ascending = False).show(5)

+-------+-----+
|page_id|event|
+-------+-----+
|     25|    2|
|     49|    2|
|     34|    1|
+-------+-----+



In [32]:
df_log2 = df_log1.select(*[i for i in df_log1.columns if i != "timestamp"], F.from_unixtime("timestamp").alias("time"))

df_log2.show()


+---+------+-------+-----+-----+-------------------+
| id|  type|page_id|  tag| sign|               time|
+---+------+-------+-----+-----+-------------------+
|234| visit|     34|  fun|false|2022-12-09 12:13:46|
|234|scroll|     34|  fun|false|2022-12-09 12:13:57|
|235| visit|     45| news|false|2022-12-09 12:15:26|
|235| click|     49|today|false|2022-12-09 12:15:41|
|235|scroll|     49|today| true|2022-12-09 12:16:07|
|235|  move|     13|sport|false|2022-12-09 12:17:23|
|236| visit|      1| main|false|2022-12-09 12:18:06|
|236|scroll|      1| main|false|2022-12-09 12:18:51|
|237| visit|     45| news|false|2022-12-09 20:40:26|
|237|scroll|     45| news|false|2022-12-09 20:41:19|
|237| click|     25| kino|false|2022-12-09 20:42:21|
|237|scroll|     25| kino| true|2022-12-09 20:43:12|
|237|  move|      1| main|false|2022-12-09 20:43:33|
|238| visit|     34|  fun|false|2022-12-09 20:47:01|
|238|scroll|     34|  fun|false|2022-12-09 20:47:57|
|238| click|     25| kino|false|2022-12-09 20:

In [13]:
df_log3 = df_log2.withColumn("time_rang", F.floor(F.hour("time") / F.lit(4)))

df_log3.show()

+---+------+-------+-----+-----+-------------------+---------+
| id|  type|page_id|  tag| sign|               time|time_rang|
+---+------+-------+-----+-----+-------------------+---------+
|234| visit|     34|  fun|false|2022-12-09 12:13:46|        3|
|234|scroll|     34|  fun|false|2022-12-09 12:13:57|        3|
|235| visit|     45| news|false|2022-12-09 12:15:26|        3|
|235| click|     49|today|false|2022-12-09 12:15:41|        3|
|235|scroll|     49|today| true|2022-12-09 12:16:07|        3|
|235|  move|     13|sport|false|2022-12-09 12:17:23|        3|
|236| visit|      1| main|false|2022-12-09 12:18:06|        3|
|236|scroll|      1| main|false|2022-12-09 12:18:51|        3|
|237| visit|     45| news|false|2022-12-09 20:40:26|        5|
|237|scroll|     45| news|false|2022-12-09 20:41:19|        5|
|237| click|     25| kino|false|2022-12-09 20:42:21|        5|
|237|scroll|     25| kino| true|2022-12-09 20:43:12|        5|
|237|  move|      1| main|false|2022-12-09 20:43:33|   

In [34]:
df_log3.groupby("time_rang").agg(F.count("*").alias("events")).orderBy("events", ascending = False).show(1)

+---------+------+
|time_rang|events|
+---------+------+
|        3|    17|
+---------+------+
only showing top 1 row



In [18]:
schema_lk = T.StructType([
                T.StructField("id", T.IntegerType(), True),
                T.StructField("user_id", T.IntegerType(), True),
                T.StructField("fio", T.StringType(), True),
                T.StructField("birthday", T.DateType(), True),
                T.StructField("lk", T.DateType(), True)])

In [21]:
data_lk = [
    (1, 234, "Иванов Петр Семенович", datetime(1999, 3, 23), datetime(2022, 11, 12)),
    (2, 235, "Боголюбов Иван Алексеевич", datetime(1983, 8, 22), datetime(2022, 11, 12)),
    (3, 236, "Прижевало Борис Алексеевич", datetime(1995, 11, 13), datetime(2022, 11, 12)),
    (4, 237, "Бойко Андрей Григорьевич", datetime(1969, 3, 21), datetime(2022, 11, 12)),
    (5, 238, "Белгородский Илья Иванович", datetime(1994, 5, 17), datetime(2022, 11, 12)),
    (6, 239, "Прижевало Борис Алексеевич", datetime(1981, 1, 1), datetime(2022, 11, 12)),
    (7, 240, "Белгородский Илья Иванович", datetime(1997, 6, 24), datetime(2022, 11, 12)),
    (8, 241, "Белгородский Сергей Иванович", datetime(1995, 8, 27), datetime(2022, 11, 12)),
]

df_lk = spark.createDataFrame(data = data_lk, schema = schema_lk)
df_lk.show()

+---+-------+--------------------+----------+----------+
| id|user_id|                 fio|  birthday|        lk|
+---+-------+--------------------+----------+----------+
|  1|    234|Иванов Петр Семен...|1999-03-23|2022-11-12|
|  2|    235|Боголюбов Иван Ал...|1983-08-22|2022-11-12|
|  3|    236|Прижевало Борис А...|1995-11-13|2022-11-12|
|  4|    237|Бойко Андрей Григ...|1969-03-21|2022-11-12|
|  5|    238|Белгородский Илья...|1994-05-17|2022-11-12|
|  6|    239|Прижевало Борис А...|1981-01-01|2022-11-12|
|  7|    240|Белгородский Илья...|1997-06-24|2022-11-12|
|  8|    241|Белгородский Серг...|1995-08-27|2022-11-12|
+---+-------+--------------------+----------+----------+



In [52]:
df_lk.alias("lk").join(df_log1.alias("web"),
            on = [F.col("lk.user_id") == F.col("web.id")], how = "left").select("fio", "tag").where((F.col('tag')=='sport')).show()   




+--------------------+-----+
|                 fio|  tag|
+--------------------+-----+
|Боголюбов Иван Ал...|sport|
|Белгородский Илья...|sport|
+--------------------+-----+

